## Top2Phase: Contrastive Learning of Water Phase from Local Topology using Edge-Conditioned Convolutional Graph Network

This colab demonstrates a simple case in which I demonstrate ideas used in the Top2Phase [paper](https://console.cloud.google.com/storage/browser/simclr-checkpoints-tf2/simclrv2/pretrained). 

* The problem defined as follow: given a set of three node forming an isosceles triangle, learn a representation for them. 

* The representation is invariant to scaling of the side length. 

* During the training we use a uniform sampling of angle, and testing for clustering is done over discrete case (training on the discrete data set will also give the same results).

In [1]:
! pip install spektral==1.0.4

     |████████████████████████████████| 122kB 5.0MB/s 


In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
from spektral.data import Dataset, DisjointLoader, Graph
from tensorflow.keras.layers import Dense, Input, ReLU, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf

from spektral.transforms.normalize_adj import NormalizeAdj
from spektral.data import BatchLoader
from spektral.layers import ECCConv, GlobalSumPool, GlobalAvgPool, GlobalAttnSumPool, GlobalMaxPool
import requests
import spektral
from tensorflow.keras.losses import Loss 
from spektral.layers import ECCConv, GlobalSumPool, GlobalAvgPool, GCNConv
print(spektral.__version__)

1.0.4


In [3]:
################################################################################
# LOAD DATA
################################################################################
class MyDataset(Dataset):
    """
    A dataset of Isosceles Triangle Represented by Graphs.
    The task is to learn a representation without labels, which is consistent with
    the angle of traingle. To do so, we use constrative learning with edge information.
    We permute graphs by stochastic scaling of orignal triangle edges. 
    we show efficiency of such representations by comparing clustering results of
    encoded representation, which turns out consistent with labels, without knowing 
    labels.
    """
    def __init__(self, n_samples, mode_cosine='disceret', **kwargs):
        self.n_samples = n_samples
        self.mode_cosine = mode_cosine
        super().__init__(**kwargs)

    def read(self):
        def make_graph():
            x = np.zeros((3,2))
            x[0,0] = 1.0
            x[1:,1] = 1.0
            a = np.ones((3,3)) - np.eye(3)
            a = a.astype(int)
            if self.mode_cosine == 'uniform':
              cost = np.random.uniform(-1,1,(1,))
            elif self.mode_cosine == 'disceret':
              cost = np.array([np.random.choice([-0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75])])#np.random.uniform(-1,1,(1,)) #
            else:
              raise NameError("Pick the right name for cosine sampling! Or implement it!")
            r1 = np.random.uniform(0.6,1.4, 1)
            r2 = r1
            e = np.zeros((3,3,1))
            e[0,1] = r1
            e[0,2] = r2
            e[1,0] = r1
            e[2,0] = r2
            e[1,2] = (r1**2+r2**2-2*cost[0]*r1*r2)**0.5
            e[2,1] = e[1,2]
            e = np.concatenate([e,np.random.uniform(0.8,1.2,1)*e],axis=-1)
            e = e.reshape((3,3,2))
            y = cost #np.array([cost[0],r1,r2]).reshape((-1))
            return Graph(x=x, a=a, e=e, y=y)
        # We must return a list of Graph objects
        return [make_graph() for _ in range(self.n_samples)]



In [4]:
tf.keras.backend.clear_session()

from tensorflow.keras import regularizers

Output_dim = 64
Latent_dim = 8
Node_dim = 2
Edge_dim = 1
Weight_decay = 0.00005 # if you want to regulairze layers use it, works fine without it
Dropout_rate = 0.01
Kernel_network =[10,10]
X_in = Input(shape=(None, Node_dim))
A_in = Input(shape=(None, None))
E_in = Input(shape=(None, None, Edge_dim))

X_1 = ECCConv(4,  kernel_network=Kernel_network, activation="relu")([X_in, A_in, E_in])
X_2 = ECCConv(8, kernel_network=Kernel_network, activation="relu")([X_1, A_in, E_in])
X_3 = ECCConv(16, kernel_network=Kernel_network, activation="relu")([X_2, A_in, E_in])
X_4 = GlobalAttnSumPool()(X_3)

X_4normalized =  tf.math.l2_normalize(X_4, axis=1, name='normalized') 

X_5 = Dense(Output_dim, activation="relu", name='projector_in')(X_4normalized)
X_5d = tf.keras.layers.Dropout(rate=Dropout_rate, name='droupout')(X_5)
X_6 = Dense(Output_dim, use_bias=False, name='projector_head')(X_5d)

model = Model(inputs=[X_in, A_in, E_in], outputs=X_6)


In [5]:
lr = 0.005
batch_size = 64
epochs = 50
samples = 1280
dataset = MyDataset(samples,mode_cosine='uniform', transforms=NormalizeAdj())
loader_tr = BatchLoader(dataset, batch_size=batch_size, epochs=epochs)
opt = Adam(lr=lr)

In [6]:
loader_tr.tf_signature()

((TensorSpec(shape=(None, None, 2), dtype=tf.float64, name=None),
  TensorSpec(shape=(None, None, None), dtype=tf.float64, name=None),
  TensorSpec(shape=(None, None, None, 2), dtype=tf.float64, name=None)),
 TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))

In [8]:
global Temp 
Temp = 0.6

@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        #print(inputs[0])
        e1 = inputs[2][:,:,:,0]
        e2 = inputs[2][:,:,:,1]
        #e1, e2 = e[::2], e[1::2]
        zi = model([inputs[0], inputs[1], e1], training =True)
        zj = model([inputs[0], inputs[1], e2], training =True)
        #zj = tf.stop_gradient(zj)
        zi = tf.math.l2_normalize(zi, axis=1)
        zj = tf.math.l2_normalize(zj, axis=1)
        zi = tf.reshape(zi, (-1,zi.shape[1]))
        zj = tf.reshape(zj, (-1,zi.shape[1]))
        z = tf.cast(tf.concat((zi, zj), 0), dtype=tf.float32)
        loss = 0.0
        tau = Temp
        batch_size = tf.shape(zi)[0]
        for k in range(batch_size):
            # Numerator (compare i,j & j,i)
            i = k
            j = k + batch_size
            # Instantiate the cosine similarity loss function
            cosine_sim = tf.keras.losses.CosineSimilarity(axis=-1, reduction=tf.keras.losses.Reduction.NONE)
            #cosine_sim = tf.keras.losses.MeanSquaredError(axis=-1, reduction=tf.keras.losses.Reduction.NONE)
            sim = tf.squeeze(- cosine_sim(tf.reshape(z[i], (1, -1)), tf.reshape(z[j], (1, -1))))
            numerator = tf.math.exp(sim / tau)

            # Denominator (compare i & j to all samples apart from themselves)
            sim_ik = - cosine_sim(tf.reshape(z[i], (1, -1)), z[tf.range(batch_size) != i])
            sim_jk = - cosine_sim(tf.reshape(z[j], (1, -1)), z[tf.range(batch_size) != j])
            denominator_ik = tf.reduce_sum(tf.math.exp(sim_ik / tau))
            denominator_jk = tf.reduce_sum(tf.math.exp(sim_jk / tau))

            # Calculate individual and combined losses
            loss_ij = - tf.math.log(numerator / denominator_ik)
            loss_ji = - tf.math.log(numerator / denominator_jk)
            #print(type(loss_ij), type(denominator_jk))
            loss += tf.add(loss_ij , loss_ji)
        
        # Divide by the total number of samples
        loss /= tf.cast(batch_size,tf.float32)
    gradients = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

print("Fitting model")
current_batch = 0
model_loss = 0
current_epoch = 0
for batch in loader_tr: 
    inps, targets = batch
    outs = train_step(*batch)
    model_loss += outs
    current_batch += 1
    if current_batch % (loader_tr.steps_per_epoch)  == 0:
        print("Epoch: {} , Loss: {} ".format(current_epoch, model_loss / loader_tr.steps_per_epoch))
        model_loss = 0
        current_epoch +=1 
        current_batch = 0

Fitting model
Epoch: 0 , Loss: 7.102535247802734 
Epoch: 1 , Loss: 6.262664318084717 
Epoch: 2 , Loss: 6.094189643859863 
Epoch: 3 , Loss: 5.951582908630371 
Epoch: 4 , Loss: 5.902822494506836 
Epoch: 5 , Loss: 5.828078269958496 
Epoch: 6 , Loss: 5.804168224334717 
Epoch: 7 , Loss: 5.780106067657471 
Epoch: 8 , Loss: 5.765320777893066 
Epoch: 9 , Loss: 5.727412223815918 
Epoch: 10 , Loss: 5.776583194732666 
Epoch: 11 , Loss: 5.789846897125244 
Epoch: 12 , Loss: 5.720215797424316 
Epoch: 13 , Loss: 5.730053424835205 
Epoch: 14 , Loss: 5.697445869445801 
Epoch: 15 , Loss: 5.658745765686035 
Epoch: 16 , Loss: 5.668644428253174 
Epoch: 17 , Loss: 5.650498867034912 
Epoch: 18 , Loss: 5.625554084777832 
Epoch: 19 , Loss: 5.646111488342285 
Epoch: 20 , Loss: 5.646829128265381 
Epoch: 21 , Loss: 5.649102210998535 
Epoch: 22 , Loss: 5.671175003051758 
Epoch: 23 , Loss: 5.612756252288818 
Epoch: 24 , Loss: 5.613945960998535 
Epoch: 25 , Loss: 5.590141296386719 
Epoch: 26 , Loss: 5.57857036590576

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 2)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, None)] 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
ecc_conv (ECCConv)              (None, None, 4)      206         input_1[0][0]                    
                                                                 input_2[0][0]                

In [10]:
dataset = MyDataset(1280, transforms=NormalizeAdj())
loader_tr = BatchLoader(dataset, batch_size=dataset.n_samples, epochs=1)
for b in loader_tr:
  inps, targets = b
model_enc = Model(inputs=model.input, outputs=model.get_layer('tf.math.l2_normalize').output)


print(inps[2].shape)

e1 = inps[2][:,:,:,0].reshape((-1,3,3,1))
e2 = inps[2][:,:,:,1].reshape((-1,3,3,1))
z = model_enc([inps[0],inps[1],e1])
z2 =  model_enc([inps[0],inps[1],e2])
for i in range(5):
  print(np.dot(z[0], z[i]), targets[0], targets[i], np.unique(e1[i]),np.unique(e2[i]))

(1280, 3, 3, 2)
0.9999998 [0.] [0.] [0.         1.23993661 1.75353517] [0.         1.33024081 1.8812446 ]
0.5680334 [0.] [-0.75] [0.         0.81316549 1.52129332] [0.         0.68817386 1.28745541]
0.9998236 [0.] [0.] [0.         1.27421765 1.80201588] [0.         1.27604498 1.80460012]
0.93104684 [0.] [-0.25] [0.        1.2595591 1.9915378] [0.         1.45054998 2.2935209 ]
0.6112633 [0.] [0.75] [0.         0.96848572 1.36964564] [0.         1.10505663 1.56278607]


In [15]:
from sklearn.cluster import AgglomerativeClustering
n_clusters=7
clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity="cosine",linkage='single').fit_predict(z/np.linalg.norm(z,1).reshape((-1,1)))
clustering[:5]

array([5, 6, 5, 2, 1])

In [16]:
print("real clusters means: ", np.unique(targets))
print("cluster:   c ,  mean(anlge[c]), std(angle[c]), boolen(|angle[c]| == |real(angle)|)")
for i in range(n_clusters):
  idx = np.where(clustering==i)[0]
  print("cluster : ", i, ",  " ,np.mean(targets[idx]), "        ,    " , np.std(targets[idx]), "       ,   " , idx.shape[0]/clustering.shape[0] == np.where(np.abs(targets-np.mean(targets[idx]))<0.02)[0].shape[0]/clustering.shape[0])#, targets[idx]>)#, np.mean(inps[2][idx]))

real clusters means:  [-0.75 -0.5  -0.25  0.    0.25  0.5   0.75]
cluster:   c ,  mean(anlge[c]), std(angle[c]), boolen(|angle[c]| == |real(angle)|)
cluster :  0 ,   -0.5         ,     0.0        ,    True
cluster :  1 ,   0.75         ,     0.0        ,    True
cluster :  2 ,   -0.25         ,     0.0        ,    True
cluster :  3 ,   0.25         ,     0.0        ,    True
cluster :  4 ,   0.5         ,     0.0        ,    True
cluster :  5 ,   0.0         ,     0.0        ,    True
cluster :  6 ,   -0.75         ,     0.0        ,    True


* The clusters are accurate and consistent with the original labels. 
* The network might be large or small, I didn't optimize it further.
* Feel free to experiment with number of cluster and networks architecture.